In [1]:
import sys, os, copy, time
sys.path.append('/home/achint/Generative_Modelling/Ziyi_2/POISEVAE-main')

import random
import numpy as np
import torch
HOME_PATH = os.path.expanduser('~')

import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import poisevae
from poisevae.datasets import MNIST
# from poisevae.networks.PixelCNN_MNIST import EncMNIST, DecMNIST
from poisevae.networks.vanilla_MNIST import EncMNIST, DecMNIST

# from poisevae.networks.pixelcnn import PixelCNN

In [2]:
MNIST_PATH = "/home/achint/datasets/MNIST/%s.pt"

joint_dataset_train = MNIST(mnist_pt_path=MNIST_PATH % 'training')
joint_dataset_test = MNIST(mnist_pt_path=MNIST_PATH % 'test')

In [3]:
batch_size = 2
train_loader = torch.utils.data.DataLoader(joint_dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(joint_dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)
len(train_loader), len(test_loader)

(30000, 5000)

In [5]:
lat1, lat2 = 20, 20
color_level = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
enc_mnist1 = EncMNIST(lat1).to(device)
dec_mnist1 = EncMNIST(lat1).to(device)
# dec_mnist1 = DecMNIST(PixelCNN(lat1, 1, color_level), color_level).to(device)

enc_mnist2 = EncMNIST(lat1).to(device)
# dec_mnist2 = DecMNIST(PixelCNN(lat2, 1, color_level), color_level).to(device)
dec_mnist2 = DecMNIST(lat2).to(device)

# Options: 'derivative_autograd', 'derivative_gradient', and 'std_normal'
vae = poisevae.POISEVAE([enc_mnist1, enc_mnist2], [dec_mnist1, dec_mnist2], latent_dims=[lat1, (lat2)], 
                        enc_config='nu', KL_calc='std_normal', batch_size=batch_size
                       ).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
writer = None

In [8]:
epoch =0
epochs = 1 + epoch
for epoch in tqdm(range(epoch, epochs)):   
    for i, data in enumerate(train_loader):
#         results = vae([data[0].to(device, dtype=torch.float32), data[0].to(device, dtype=torch.float32)], dec_kwargs={'generate_mode':False})#, 
                      # n_gibbs_iter=30) 
        results = vae([data[0].to(device, dtype=torch.float32), data[0].to(device, dtype=torch.float32)])#, 
                      # n_gibbs_iter=30)    
# with torch.no_grad():
#     for i, data in enumerate(test_loader):
#         results = vae([data[0].to(device, dtype=torch.float32), data[0].to(device, dtype=torch.float32)], dec_kwargs={'generate_mode':False})#, 
#                       # n_gibbs_iter=30)
#         break

  0%|                                                     | 0/1 [00:00<?, ?it/s]


TypeError: forward() takes 2 positional arguments but 3 were given

In [ ]:
#results

In [ ]:
x_rec = results['x_rec']

if isinstance(x_rec[0], torch.distributions.Distribution):
#     x_rec = [x_rec[0].probs, x_rec[1].probs]
#     idx = torch.where(results['weights'] == results['weights'].max(dim=1).values[:, None])
#     x_rec = [x_rec[0].loc[idx], x_rec[1].loc[idx]] # (batch, IW samples, ...)
    x_rec = [x_rec[0].loc[idx], x_rec[1].loc[idx]] # (batch, IW samples, ...)

In [ ]:
ncols = min(len(x_rec[0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(data[0], x_rec[0][:,1,:,:,:])):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_xticks([])
        ax[j, i].set_yticks([])
ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)
# fig.savefig(PATH + 'MNISTRec.pdf', dpi=300)

In [ ]:
x_rec[0].size()

In [ ]:
data[0].size()

In [ ]:
x_rec1 = results['x_rec'][0].max(dim=1).indices.squeeze()
x_rec1.size()

In [ ]:
x_rec0 = x_rec[0].max(dim=1).indices.squeeze()
x_rec1 = x_rec[1].max(dim=1).indices.squeeze()
x_rec = [x_rec0, x_rec1]

In [ ]:
ncols = min(len(x_rec[0]), 20)
fig, ax = plt.subplots(nrows=2, ncols=ncols, figsize=(15, 1.5))
for i, aux in enumerate(zip(data[0], x_rec[0][:,5,:,:,:])):
    if i >= ncols:
        break
    for j, im in enumerate(aux):
        ax[j, i].imshow(im.cpu().numpy().reshape(28, 28), cmap='gray')
        ax[j, i].set_xticks([])
        ax[j, i].set_yticks([])
ax[1, 0].set_ylabel('Rec', fontsize=24)
fig.tight_layout(pad=0)